In [27]:
import requests
from pprint import pprint
import os
import json
from pandas import DataFrame 
from dotenv import load_dotenv
import pandas as pd
from bs4 import BeautifulSoup
import re
import time


In [54]:
load_dotenv()
# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

print(f"{client_id}, {client_secret}")

NQwLlbDgiq1GTGqk2dyA, 4oWbLLDKc2


In [99]:
import os
import time
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup

def find_nhn_book(query: str):
    url = "https://openapi.naver.com/v1/search/book.json"

    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret,
        'User-Agent': 'Mozilla/5.0'
    }

    payload = {
        'query': query,
        'display': 50,
        'sort': 'sim'
    }

    res = requests.get(url=url, headers=headers, params=payload)

    if not res.ok:
        print(f"API 요청 실패: {res.status_code}")
        return None

    items = res.json()['items']
    book_df = DataFrame(items)
    print(book_df['link'])

    item_price_list = []
    for link in book_df['link']:
        time.sleep(1.5)
        print(link)
        res_link = requests.get(url=link, headers=headers)
        print(res_link.status_code)

        if res_link.ok:
            soup = BeautifulSoup(res_link.content, 'html.parser')
            price_tag = soup.select_one(
                'div[class*=bookSeller_book_seller] > div:nth-child(1)> span[class=bookSeller_price__kPMmd] em'
            )
            item_price_list.append(
                price_tag.text.replace(',', '') if price_tag else '0'
            )
        else:
            print(f"링크 통신 오류: {res_link.status_code}")
            break

    final_df = book_df.iloc[:len(item_price_list)].copy()
    final_df['price'] = item_price_list

    os.makedirs('data', exist_ok=True)
    json_file_name = 'data/books.json'
    final_df.to_json(json_file_name, force_ascii=False)
    print(f"{json_file_name} saved")

    return final_df


In [100]:
a = find_nhn_book ('파이썬')

0     https://search.shopping.naver.com/book/catalog...
1     https://search.shopping.naver.com/book/catalog...
2     https://search.shopping.naver.com/book/catalog...
3     https://search.shopping.naver.com/book/catalog...
4     https://search.shopping.naver.com/book/catalog...
5     https://search.shopping.naver.com/book/catalog...
6     https://search.shopping.naver.com/book/catalog...
7     https://search.shopping.naver.com/book/catalog...
8     https://search.shopping.naver.com/book/catalog...
9     https://search.shopping.naver.com/book/catalog...
10    https://search.shopping.naver.com/book/catalog...
11    https://search.shopping.naver.com/book/catalog...
12    https://search.shopping.naver.com/book/catalog...
13    https://search.shopping.naver.com/book/catalog...
14    https://search.shopping.naver.com/book/catalog...
15    https://search.shopping.naver.com/book/catalog...
16    https://search.shopping.naver.com/book/catalog...
17    https://search.shopping.naver.com/book/cat

In [101]:
a.columns

Index(['title', 'link', 'image', 'author', 'discount', 'publisher', 'pubdate',
       'isbn', 'description', 'price'],
      dtype='object')

In [135]:
#3. 질문 :  검색어로  찾은  책 목록 출력하기
a.head()

,title,link,image,author,discount,publisher,pubdate,isbn,description,price
0,처음이야? 파이썬 기초 (동영상 강의로 배우는 292개 코드 따라하기(핵심노트+오픈...,https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_550256...,윤영빈^오환^이용희,18000,영진닷컴,20250620,9788931478006,파이썬 완전 초보에게 추천합니다.\n파이썬을 처음 배우는 당신을 위한 가장 쉬운 기...,18000
1,모두의 인공지능 with 파이썬 (누구나 쉽게 시작하는 AI 기초 프로그래밍),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_558136...,이영호,27000,길벗,20250725,9791140714995,"가장 쉬운 인공지능 입문서, \n생성형 AI를 반영해 한층 더 탄탄하게 돌아왔다!\...",27000
2,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_325076...,윤인성,19800,한빛미디어,20220601,9791162245651,혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서\n\n『혼자 ...,19800
3,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_403540...,박응용,19800,이지스퍼블리싱,20230615,9791163034735,프로그래밍 분야 8년 연속 베스트셀러!\n《Do it! 점프 투 파이썬》 전면 개정...,19800
4,파이썬,https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_324609...,홍의경,30400,생능출판,20220309,9788970506784,코딩 실력을 키우는 지름길은 실습이다.\n\n2008년 즈음에 검인정 중학교 교과서...,30400


In [136]:
#4. 질문 :  검색어로  찾은  책 목록 중에서 가격이 2만원 이상인 책만 출력하기
a.loc[a['price'].astype(int)>20000, ['title','author','discount','publisher','pubdate']].copy()

,title,author,discount,publisher,pubdate
1,모두의 인공지능 with 파이썬 (누구나 쉽게 시작하는 AI 기초 프로그래밍),이영호,27000,길벗,20250725
4,파이썬,홍의경,30400,생능출판,20220309
9,파이썬,김영천,23750,기한재,20200225
12,파이썬 (다양한 예제로 알로리즘 쉽게 분석하기),김유진,22500,다본,20190410
13,파이썬,Y. Daniel Liang,31500,에피스테메,20180302
15,밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현),사이토 고키,23400,한빛미디어,20250124
16,독학 파이썬,야마다 요시히로,28800,정보문화사,20241220
18,코딩 자율학습 나도코딩의 파이썬 입문 (초보자 눈높이에 맞춘 친절한 프로그래밍 자습서),나도코딩,20520,길벗,20230220
19,내공의 파이썬 (한 권으로 끝내는 초보자용 코딩 입문서),한선관,24300,자유아카데미,20250315
21,파이썬 + AI (코드조각과 실행화면으로 배우는 코딩),안용제^김남규,23940,디지털북스,20240715


In [137]:
#6. 질문 :  검색어로  찾은  책 목록 중에서 출판사가 인피니티북스인 책만 출력하기
a.loc[a['publisher'] == '인피니티북스'].drop(columns=['image' , 'description'])

,title,link,author,discount,publisher,pubdate,isbn,price
6,파이썬,https://search.shopping.naver.com/book/catalog...,천인국,10800,인피니티북스,20170830,9791185578330,10800


,title,link,image,author,discount,publisher,pubdate,isbn,description,price
6,파이썬,https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_324891...,천인국,10800,인피니티북스,20170830,9791185578330,컴퓨팅 사고(Computational Thinking)는 자넷 윙 교수가 2006년...,10800


In [ ]:

from dotenv import load_dotenv
# .env 파일에서 환경 변수 로드
load_dotenv()